# Data Loading

In this lab, we are going to focus on loading our data about baseball players into a database. Why use a database instead of files? Conceptually, we do this when we want to enforce rules on the structure of the data so that issues of cleanliness, inconsistency and missing data are identified prior to our attempts to do analysis. 

Database management systems provide well defined structure for data. They also have the advantage of giving us standard mechanisms for extracting data: "Structured Query Language", or "SQL. If  you have not used SQL before, it will require a little adjustment. Once you are familiar with it, however, you will find SQL intuitive and portable.

As you learned in the previous lab, Data Carpentry is often required to transform your messy data into a useable structure. 

Data Carpentry is a fancier, more elegant term for "Data Janitorial" work. Most of us would rather be carpenters than janitors. If you spend any time around children, you recognize that life *requires* more janitorial work than carpentry. And so it is with data science. But carpentry *sounds* [1] better, so we're going to use that [2]. 

Using a database allows you to store your transformed and cleaned data into a reusable, structured, and semantically labelled format.

You can then access this data in the future using structured query language (SQL). 

## Procedure

1. Inspect data, develop semantically structured data storage (i.e., database schema)
2. Develop data transformations, cleaning, and re-organizations
3. Push data into the database

__NOTE:__ A quick review of the *Database and SQL Bootcamp* may be useful at this time.
In JupyterHub, this was deployed for you as **BC-SQL**.

----

## Inspect

For this lab, we are going to use relatively clean data that is in nice comma separated values (CSV) format.
Typically, the data requires data carpentry activities, but for the sake of a more simple collection of samples and discussion we are going to start with easy data.

To work with our data files we are going to use Pandas and Numpy

[1]: https://www.youtube.com/watch?v=pMRUszqMVM8 "Replacements: Waitress in the Sky Song"
[2]: http://www.azlyrics.com/lyrics/replacements/waitressinthesky.html "Replacements: Waitress in the Sky Lyrics"

In [ ]:
import pandas as pd
import numpy as np
players = pd.read_csv('../../../datasets/baseball-databank/data/Master.csv')
teams = pd.read_csv('../../../datasets/baseball-databank/data/Teams.csv')
batting = pd.read_csv('../../../datasets/baseball-databank/data/Batting.csv')

__No Output Expected__

Now we have loaded our three files into the variables: *players*, *teams*, and *batting*.
Each of these variables is a Pandas **dataframe**.
As you have often done before, we can preview the data with the *head()* method called on the **dataframe** variable.

In [ ]:
players.head()

In [ ]:
teams.head()

In [ ]:
batting.head()

----
As we can see, the CSV files are tabluar data files. 
Note, in each case the tables cannot fit within the display and the *ellipse* (...) is used to denote columns that are removed from the display.
Do you recall how to view the columns of a dataframe?
Like all things python, there are a few ways to do this.
We will just use the list function to inspect the dataframe.

In [ ]:
list(players)

We get to see all the columns of the dataframe this way

__Your Turn__  
In the cell below, add code to list the columns of the teams and batting

In [ ]:
## Add your code in this cell





### We know the columns... now what?

## Database Design

Now that we know the columns, we need to contemplate how we will use a database to structure the data.
Recall from your previous database knowledge, or the bootcamp, that a relational database is an organized set of __tables__ (aka *Relations*).

Recall that __tables__ are a structured set of columns with semantic meaning, a particular data type, and constraints on validity.
We are assuming that you have a little domain knowledge in Baseball to interpret the semantics of these column labels.
For example, the *batting* column *RBI* we can expect to be *Runs Batted In*.

#### REVIEW : SQL : Create Table
```SQL
CREATE TABLE table_name (
  col_a_name col_a_datatype, 
  col_b_name col_b_datatype, 
  col_c_name col_c_datatype, 
  ...
  PRIMARY KEY(list_of_columns)
);
```
**REFERENCE LINK** [SQLite Create Table](https://www.sqlite.org/lang_createtable.html)

### Column Data Types

Databases support very rigid data typing, however SQLite permits looser *type affiinty* based on storage classes.
From the SQLite documentation: 
```
Each column in an SQLite 3 database is assigned one of the following type affinities:
    TEXT
    NUMERIC
    INTEGER
    REAL
    BLOB
```

For this exercise, we will limit our storage columns to one of:
 1. **TEXT** - Character strings
 2. **INTEGER** - whole numbers, no decimal places
 3. **REAL** - floating point numbers with decimal places


**REFERENCE LINK** [SQLite Column Data Types](https://www.sqlite.org/datatype3.html)

So, how can we exam in a programmatic way the data types as interpreted by Pandas?  
Recall that a dataframe provides column access via __dataframe__[*column_name*], and the column is an object that holds a list of values and the data type.

In [ ]:
# dtype is the Data Type of the column that is referenced by name in the square brackets
players['birthYear'].dtype

We see that bithYear is a floating point number, which above we refer to as a **REAL**

Since we have python at our fingertips... lets programmatically inspect the colunms and data types

In [ ]:
# Remember above, the list command did an inspection and 
# got a list of column names in the data frame!
for columnName in list(players):
    print("Column {} is a {}".format(columnName, players[columnName].dtype))


The output above should look similar to:
```
Column playerID is a object
Column birthYear is a float64
...
Column bbrefID is a object
```

The **object** datatype we will interpret as **TEXT** for the data base.
Scroll back up to when we previewed the data files, does this seem reasonable?  
Let us check, just to be sure

In [ ]:
players['playerID'].head()

Seems OK!
You should typically check every column that you are going to load into the database.

Once you are ready to create a table, we can write the create table statement:

```SQL
CREATE TABLE players (
  playerID TEXT,
  birthYear REAL,
  ...
);
```

Can we automate this for a generic CSV to SQL Table?

----

This segment of code shows the use of Python to automate the generation of SQL Create table from a Pandas dataframe.

__Note:__ the special escape characters for *newline* (''\n'') and *tab* (''\t'') are used to generate visually pleasing SQL, they are not required.

In [ ]:
# Begin the create table statement
createTableStmt = "CREATE TABLE players (\n"

# Build a translate from Panda to SQL type
dtype2SQL = {'object' : 'TEXT', 'float64' : 'REAL', 'int64' : "INTEGER"}
# Note, the int64 ?  That came from the teams and batting dataframes


columnList = list(players)

for columnName in columnList:
    pandaType = str(players[columnName].dtype) # Note, we need to force the conversion of the type name to a string
    sqlDataTypeStr = dtype2SQL[pandaType]      # Then we look up the SQL type Desired
    #
    #  Construct a Column Spec 
    #  col_name col_dtype , 
    createTableStmt += "\t{} {},\n".format(columnName, sqlDataTypeStr)
    #
    # NOTE:  the string1 += string2 appends string2 to the end of string 1, e.g., "ABC"+="XYZ" results in "ABCXYZ"
    #
    
    
# Note, the last column has a trailing comma, so we can now add a Primary Key specification
# If this is not suitable for the data file you have, you will need to make adjustments 
# such as removing the last comma before closing off the table.
createTableStmt += "\tPRIMARY KEY({})\n".format(columnList[0])



# Close off the Create Table Statement
createTableStmt += ");"

print(len(createTableStmt))

In [ ]:
###################################
## 
## Now lets modularize this:
##
###################################
def dataframe2CreateTable(dataFrame, tableName = "WHATS_MY_NAME",useFirstColumnAsPK=True):
    '''
    This function inspects a Panda Dataframe and converts it to 
    a SQL Create Table Statement String
    
    Arguments:
       dataFrame : a panda dataframe with column headers
       tableName : a valid SQL table name
       useFirstColumnAsPK : Use the first column as a primary key, default=True
    
    Returns : a Create Table tableName string
    '''
    createTableStmt = "CREATE TABLE {} (\n".format(tableName)  # used the format to splice in the table name 
    dtype2SQL = {'object' : 'TEXT', 'float64' : 'REAL', 'int64' : "INTEGER"}
    columnList = list(dataFrame)  # Replaced players from code with function variable
    
    for columnName in columnList:
        # NOTE: Some of the columns start with a number, this is not valid column naming
        # in most databases;  so the next four lines detect and fix
        if (columnName[0].isdigit()):
            sqlColumnName = "n"+columnName   # we will just prepend the letter 'n' (for number)
        else:
            sqlColumnName = columnName

        pandaType = str(dataFrame[columnName].dtype) # Note, we need to force the conversion of the type name to a string
        sqlDataTypeStr = dtype2SQL[pandaType]      # Then we look up the SQL type Desired
        createTableStmt += "\t{} {},\n".format(sqlColumnName, sqlDataTypeStr)
    # END OF FOR EACH COLUMN
    
    # Close off the Create Table Statement with the PK
    if (useFirstColumnAsPK):
        createTableStmt += "\tPRIMARY KEY({})\n".format(columnList[0])
    else: # replace last comma with a space, note it's minus 2 because -1 is the newline
                                          # This is the substring access 
                                          # see : https://docs.python.org/3/tutorial/introduction.html#strings
        createTableStmt = createTableStmt[:len(createTableStmt) -2] + "\n"
    createTableStmt += ");"
    
    return  createTableStmt
# ------- END OF dataframe2CreateTable


# Invoke
help(dataframe2CreateTable)

### Putting our DB creation together

In [ ]:
# We are going to write a SQLite DB
import sqlite3

playersCreateTableStmt = dataframe2CreateTable(dataFrame = players, tableName = 'players')
teamsCreateTableStmt = dataframe2CreateTable(dataFrame = teams, tableName = 'teams', useFirstColumnAsPK=False)
battingCreateTableStmt = dataframe2CreateTable(dataFrame = batting, tableName = 'batting', useFirstColumnAsPK=False)


print(playersCreateTableStmt)
print(teamsCreateTableStmt)
print(battingCreateTableStmt)

**NOTE:** In reality, some of the column data types such as year and counting statitics should be INTEGER type.
However, for this example we will just move forward.
There are ways to manipulate the panda dataframe to move the data into a better aligned colunm data type.
We will leave that as a thought exercise for now.

__WARNING__ : 
Please note, that when we first connect to a database using SQLite and it does not exist, it gets created for us.  
This *friendly* behavior can be REALLY OCNFUSING on that day in the future when you have a file path wrong on a database you previous have populated and it looks empty to your code.  

__REFERENCE__ : [Python SQLite3](https://docs.python.org/3/library/sqlite3.html)


In [ ]:
import os

# Below is a pathname = ../baseball.db 
# The path is broken into elements around the '/' character (i.e., "forward slash" because it leans forward)
# The first path element is the '..'  which is interpreted as the parent directory/folder. 
#        Look at the URL.  File file is named: --- module3/labs/database_loading.ipynb
#        This file is in a folder named labs, which is in a folder named module3.
#        This path name is therefore for module3/labs/../baseball.db
#                   ... which is equivalent to module3/baseball.db
#        We are putting the file there so it is accesible during exercises
databaseFilename = '../baseball.db'

# Just because we are creating this file here
#  we will remove it incase you re-run the cell
if os.path.exists(databaseFilename):
    os.remove(databaseFilename)
    
# Open / Create the baseball.db database file.
connection = sqlite3.connect(databaseFilename)

# SQLite uses a cursor to track and manage and group operations.
cursor = connection.cursor()
# A cursor is a database execution context that provides isoation between 
#  the operations in the cursor and other operations that are happening
#  simultaneously.
# These operations can be undone by cancelling (i.e., ROLLBACK) the transaction before the cursor context 
#  is committed

# Create tables
cursor.execute(playersCreateTableStmt)
cursor.execute(teamsCreateTableStmt)
cursor.execute(battingCreateTableStmt)

# Save (commit) the changes
connection.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
connection.close()

__NO OUTPUT EXPECTED__


### Did this work?
You can either open the file using the command line as you may hav done in the bootcamp.

![Command Line Image](../images/SQLite3_baseball_db_players_schema.png)

... or we can use SQL and Python.

In [ ]:
## Did this actually work?
#  Open the DB file
databaseFilename = '../baseball.db'
connection = sqlite3.connect(databaseFilename)

# Select the list of tables from the SQLite Engine Catalog for the database file
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Iterate through all the rows back, where the first column is the table name
for table_name in tables:
    print(table_name[0])

### FINALLY ... we get to load the data

What this entails is iterating through the dataframe and inserting the values into the table

In [ ]:
databaseFilename = '../baseball.db'
connection = sqlite3.connect(databaseFilename)
cursor = connection.cursor()

for row in players.itertuples(index=False, name ='None'):
    cursor.execute('INSERT INTO players VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',row)

# Save (commit) the changes
connection.commit()


__REFERENCE:__ [itertupples : Iterate Through Dataframe Rows, each row a tuple](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples)

__How to see the data from the database prompt__
![Select 5 players](../images/SQLite_baseball_select_5_players.png)

In [ ]:
databaseFilename = '../baseball.db'
connection = sqlite3.connect(databaseFilename)
cursor = connection.cursor()

batting.fillna(value=0) # Fill NaN values

# Or stand on the shoulders including giants
cursor.executemany('INSERT INTO batting VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
                   batting.itertuples(index=False, name ='None'))


# Save (commit) the changes
connection.commit()

One of the things we have not addressed is dealing with the ''NaN'' values in many cells of the data frames.


__REFERENCE:__ Now that we did all this in a drawn out fasion, see [SQL Loading from Pandas](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html)  
See Also: [SQLAlchemy](http://www.sqlalchemy.org/)


__REMEMBER__ : This lab used clean CSV files that were mostly straight foward. Often, data carpentry activities requires the efforts of the previous lab as well as this lab.



# SAVE YOUR NOTE BOOK

